In [1]:
import scrap
client_access_token = scrap.API_TOKEN

In [3]:
import lyricsgenius
LyricsGenius = lyricsgenius.Genius(client_access_token)

In [14]:
artist_name = str(input('Enter artist name'))

In [16]:
artist = LyricsGenius.search_artist(artist_name, max_songs=6)

Searching for songs by Dua Lipa...

Song 1: "New Rules"
Song 2: "Don’t Start Now"
Song 3: "IDGAF"
Song 4: "Levitating"
Song 5: "Blow Your Mind (Mwah)"
Song 6: "Break My Heart"

Reached user-specified song limit (6).
Done. Found 6 songs.


In [17]:
for song in artist.songs:
    print(song.lyrics)

[Intro]
One, one, one, one, one

[Verse 1]
Talkin' in my sleep at night, makin' myself crazy
(Out of my mind, out of my mind)
Wrote it down and read it out, hopin' it would save me
(Too many times, too many times)

[Refrain]
My love, he makes me feel like nobody else, nobody else
But my love, he doesn't love me, so I tell myself, I tell myself

[Pre-Chorus]
One: Don't pick up the phone
You know he's only callin' 'cause he's drunk and alone
Two: Don't let him in
You'll have to kick him out again
Three: Don't be his friend
You know you're gonna wake up in his bed in the morning
And if you're under him, you ain't gettin' over him

[Chorus]
I got new rules, I count 'em
I got new rules, I count 'em
I gotta tell them to myself
I got new rules, I count 'em
I gotta tell them to myself

[Verse 2]
I keep pushin' forwards, but he keeps pullin' me backwards
(Nowhere to turn, no way)
(Nowhere to turn, no)
Now I'm standin' back from it, I finally see the pattern
(I never learn, I never learn)

[Refr

In [18]:
song = LyricsGenius.search_song(artist_name, "New Rules" )

Searching for "Dua Lipa" by New Rules...
Done.


In [19]:
song.lyrics

"[Intro]\nOne, one, one, one, one\n\n[Verse 1]\nTalkin' in my sleep at night, makin' myself crazy\n(Out of my mind, out of my mind)\nWrote it down and read it out, hopin' it would save me\n(Too many times, too many times)\n\n[Refrain]\nMy love, he makes me feel like nobody else, nobody else\nBut my love, he doesn't love me, so I tell myself, I tell myself\n\n[Pre-Chorus]\nOne: Don't pick up the phone\nYou know he's only callin' 'cause he's drunk and alone\nTwo: Don't let him in\nYou'll have to kick him out again\nThree: Don't be his friend\nYou know you're gonna wake up in his bed in the morning\nAnd if you're under him, you ain't gettin' over him\n\n[Chorus]\nI got new rules, I count 'em\nI got new rules, I count 'em\nI gotta tell them to myself\nI got new rules, I count 'em\nI gotta tell them to myself\n\n[Verse 2]\nI keep pushin' forwards, but he keeps pullin' me backwards\n(Nowhere to turn, no way)\n(Nowhere to turn, no)\nNow I'm standin' back from it, I finally see the pattern\n(I

In [20]:
song.save_lyrics(extension='txt')

Wrote lyrics_dualipa_newrules.txt.


In [21]:
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
from pathlib import Path

In [22]:
def clean_up(song_title):

    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    
    return clean_song_title

In [23]:
def get_all_songs_from_album(artist, album_name):
    
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")
    
    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
        
    return clean_songs

In [24]:
def download_album_lyrics(artist, album_name): 
    
    # Set up LyricsGenius with your Genius API client access token
    #client_access_token = Your-Client-Access-Token
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    
    # With the function that we previously created, go to Genius.com and get all song titles for a particular artist's album
    clean_songs = get_all_songs_from_album(artist, album_name)
    
    for song in clean_songs:
        
        #For each song in the list, search for that song with LyricsGenius
        song_object = LyricsGenius.search_song(song, artist)
        
        #If the song is not empty
        if song_object != None:
            
            #Do some cleaning and prep for the filename of the song
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            
            #Establish the filename for each song inside a directory that begins with the artist's name and album title
            custom_filename=f"{artist_title}_{album_title}/{song_title}"
            
            #A line of code that we need to create a directory
             #os.makedirs(os.path.dirname(filename), exist_ok=True)
            Path(f"{artist_title}_{album_title}").mkdir(parents=True, exist_ok=True)
            
            #Save the lyrics for the song as a text file
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        
        #If the song doesn't contain lyrics
        else:
            print('No lyrics')

In [25]:
download_album_lyrics(artist_name, "Dua Lipa")

Searching for "Genesis" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Genesis.txt.
Searching for "Lost in Your Light" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Lost-in-Your-Light.txt.
Searching for "Hotter Than Hell" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Hotter-Than-Hell.txt.
Searching for "Be the One" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Be-the-One.txt.
Searching for "IDGAF" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/IDGAF.txt.
Searching for "Blow Your Mind (Mwah)" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Blow-Your-Mind-(Mwah).txt.
Searching for "Garden" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Garden.txt.
Searching for "No Goodbyes" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/No-Goodbyes.txt.
Searching for "Thinking 'Bout You" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Thinking-'Bout-You.txt.
Searching for "New Rules" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/New-Rules.txt.
Searching for "Begging" by Dua Lipa...
Done.
Wrote Dua-Lipa_Dua-Lipa/Begging.t